## Equity pipeline
Use the built-in transformer to normalize trades before building equity.

In [2]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent if not (Path.cwd() / "src").exists() else Path.cwd()
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH in sys.path:", SRC_PATH in map(Path, sys.path))

PROJECT_ROOT: c:\Users\user\Documents\spreader_pro\code\test_visualisation
SRC_PATH in sys.path: True


### Создание датасета OHCL для тестового периода

In [3]:
from data.candle_loader import load_candles  # т.к. файл src/data/candle_loader.py

# укажи путь к файлу с результатами сделок (тот, где "Дата и время" и "Код инструмента")
RESULT_CSV = 'WORK_GLD_RW_trendt.csv'   # поменяй под себя

# если сырые минутки лежат у тебя в другом месте — можно переопределить
DATA_DIR = r"G:\My Drive\data_fut"  # или r"G:\My Drive\data_fut"

# если справочник шагов не рядом с модулем — тоже можно указать:
STEPS_PATH = PROJECT_ROOT / "src" / "data" / "moex_tickers_steps.csv"

df = load_candles(
    result_csv_path=str(RESULT_CSV),
    data_dir=str(DATA_DIR),
    steps_path=str(STEPS_PATH),
    resample_rule=None,   # или "5T" / "5min"
    save=True,
)

In [4]:
from data.equity_builder import build_equity

candle_csv = PROJECT_ROOT / "WORK_GLD_RW_trendt_candle_data.csv"
trades_csv = PROJECT_ROOT / "WORK_GLD_RW_trendt.csv"

equity_df = build_equity(
    candles_csv=candle_csv,
    trades_csv=trades_csv,
    trades_use_transformer=True,
)
equity_df.head()

c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.to_datetime(col, errors="coerce").dt.floor("T")
c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\csv_transformer.py:79: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df["Дата и время"] = df["Дата и время"].dt.floor("T")  # 'T' == minute
c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.to_datetime(col, errors="coerce").dt.floor("T")


,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,1_step_price,date_time,action_price,current_pos,prev_position,deal_price,prev_action_price,delta_price,Pnl,Equity
0,2025-09-04 17:14:00,9137.2,9137.2,9135.0,9135.6,325.0,1.0,2025-09-04 17:14:00,9135.0,0,0,NaN,9135.0,0.0,0.0,0.0
1,2025-09-04 17:15:00,9136.0,9136.0,9132.3,9135.2,940.0,1.0,2025-09-04 17:15:00,9134.5,-10,0,9134.5,9135.0,-0.5,-0.0,0.0
2,2025-09-04 17:16:00,9134.7,9134.7,9132.5,9132.5,229.0,1.0,2025-09-04 17:16:00,9134.7,-10,-10,NaN,9134.5,0.2,-2.0,-2.0
3,2025-09-04 17:17:00,9133.0,9135.7,9132.8,9134.1,96.0,1.0,2025-09-04 17:17:00,9135.7,-10,-10,NaN,9134.7,1.0,-10.0,-12.0
4,2025-09-04 17:18:00,9133.8,9136.2,9132.5,9132.5,230.0,1.0,2025-09-04 17:18:00,9134.2,0,-10,9134.2,9135.7,-1.5,15.0,3.0


In [8]:
trades_csv.head()

AttributeError: 'WindowsPath' object has no attribute 'head'

In [5]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergl(        # ⚡ ScatterGL — рендер на GPU
    x=equity_df['DATE_TIME'],
    y=equity_df['Equity'],
    mode='lines',
    line=dict(width=1, color='blue')
))
fig.show()




In [7]:
equity_df.tail(50)

,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,1_step_price,date_time,action_price,current_pos,prev_position,deal_price,prev_action_price,delta_price,Pnl,Equity
36042,2025-10-28 16:34:00,10005.0,10009.7,9999.9,10002.4,1443.0,1.0,2025-10-28 16:34:00,10009.7,-150,-150,NaN,10011.9,-2.2,330.0,47100.0
36043,2025-10-28 16:35:00,10000.1,10018.7,10000.0,10005.1,2974.0,1.0,2025-10-28 16:35:00,10018.7,-150,-150,NaN,10009.7,9.0,-1350.0,45750.0
36044,2025-10-28 16:36:00,10005.2,10025.5,10005.1,10025.5,533.0,1.0,2025-10-28 16:36:00,10025.5,-150,-150,NaN,10018.7,6.8,-1020.0,44730.0
36045,2025-10-28 16:37:00,10023.6,10023.6,10017.9,10017.9,436.0,1.0,2025-10-28 16:37:00,10023.6,-150,-150,NaN,10025.5,-1.9,285.0,45015.0
36046,2025-10-28 16:38:00,10016.4,10023.9,10011.4,10023.9,435.0,1.0,2025-10-28 16:38:00,10023.9,-150,-150,NaN,10023.6,0.3,-45.0,44970.0
36047,2025-10-28 16:39:00,10020.8,10022.8,10011.2,10011.4,71.0,1.0,2025-10-28 16:39:00,10022.8,-150,-150,NaN,10023.9,-1.1,165.0,45135.0
36048,2025-10-28 16:40:00,10011.4,10018.7,10006.5,10008.8,1862.0,1.0,2025-10-28 16:40:00,10018.7,-150,-150,NaN,10022.8,-4.1,615.0,45750.0
36049,2025-10-28 16:41:00,10005.2,10006.4,9986.4,9989.0,1649.0,1.0,2025-10-28 16:41:00,10006.4,-150,-150,NaN,10018.7,-12.3,1845.0,47595.0
36050,2025-10-28 16:42:00,9992.5,9999.4,9987.6,9997.4,453.0,1.0,2025-10-28 16:42:00,9999.4,-150,-150,NaN,10006.4,-7.0,1050.0,48645.0
36051,2025-10-28 16:43:00,9997.5,10011.8,9997.5,10002.7,846.0,1.0,2025-10-28 16:43:00,10011.8,-150,-150,NaN,9999.4,12.4,-1860.0,46785.0
